DDOS

In [1]:
import pandas as pd
import hashlib
import os
import json
from scapy.all import rdpcap, IP, TCP, UDP, ICMP
from datetime import datetime
from tqdm import tqdm
import numpy as np
if os.path.isfile('/home/usami/TBN64/dataset_pre/CICDDoS2017/TrafficLabelling/Friday-WorkingHours-nor.csv'):
    nor = {}
    ddos = {}
    port = {}
    bot = {}
    nor['ID'] = pd.read_csv('/home/usami/TBN64/dataset_pre/CICDDoS2017/TrafficLabelling/Friday-WorkingHours-nor.csv')
    ddos['ID'] = pd.read_csv('/home/usami/TBN64/dataset_pre/CICDDoS2017/TrafficLabelling/Friday-WorkingHours-ddos.csv')
    port['ID'] = pd.read_csv('/home/usami/TBN64/dataset_pre/CICDDoS2017/TrafficLabelling/Friday-WorkingHours-port.csv')
    bot['ID'] = pd.read_csv('/home/usami/TBN64/dataset_pre/CICDDoS2017/TrafficLabelling/Friday-WorkingHours-bot.csv')
else:
    file_path_dos = '/home/usami/TBN64/dataset_pre/CICDDoS2017/TrafficLabelling/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'
    file_path_port = '/home/usami/TBN64/dataset_pre/CICDDoS2017/TrafficLabelling/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv'
    file_path_bot = '/home/usami/TBN64/dataset_pre/CICDDoS2017/TrafficLabelling/Friday-WorkingHours-Morning.pcap_ISCX.csv'
    df_ddos = pd.read_csv(file_path_dos)
    df_port = pd.read_csv(file_path_port)
    df_bot = pd.read_csv(file_path_bot)
    saddr = ' Source IP'
    sport = ' Source Port'
    daddr = ' Destination IP'
    dport = ' Destination Port'
    proto = ' Protocol'

    label = ' Label'
    nor_label = 'BENIGN'
    bot_label = 'Bot'
    port_label = 'PortScan'
    ddos_label = 'DDoS'

    df_ddos = df_ddos[df_ddos[proto] != 0]
    df_port = df_port[df_port[proto] != 0]
    df_bot = df_bot[df_bot[proto] != 0]

    def create_id(row):
        if row[proto] in [6, 17]:  # TCP or UDP
            id_str = f"{row[saddr]}:{int(row[sport])}->{row[daddr]}:{int(row[dport])}:{row[proto]}"
        elif row[proto] == 1:  # ICMP
            id_str = f"{row[saddr]}->{row[daddr]}:1"
        return id_str
        # return int(hashlib.md5(id_str.encode()).hexdigest(), 32)
    nor = df_ddos[df_ddos[label] == nor_label]
    pd.concat([nor, df_port[df_port[label] == nor_label]], ignore_index=True)
    pd.concat([nor, df_bot[df_bot[label] == nor_label]], ignore_index=True)

    ddos = df_ddos[df_ddos[label] == ddos_label]
    port = df_port[df_port[label] == port_label]
    bot = df_bot[df_bot[label] == bot_label]

    nor['ID'] = nor.apply(create_id, axis=1)
    ddos['ID'] = ddos.apply(create_id, axis=1)
    port['ID'] = port.apply(create_id, axis=1)
    bot['ID'] = bot.apply(create_id, axis=1)
    duplicate_ddos = pd.Series(list(set(nor['ID']).intersection(set(ddos['ID']))))
    duplicate_port = pd.Series(list(set(nor['ID']).intersection(set(port['ID']))))
    duplicate_bot = pd.Series(list(set(nor['ID']).intersection(set(bot['ID']))))

    nor = nor[~nor['ID'].isin(duplicate_ddos)]
    nor = nor[~nor['ID'].isin(duplicate_port)]
    nor = nor[~nor['ID'].isin(duplicate_bot)]
    nor_id = pd.DataFrame(nor['ID'].unique())
    ddos_id = pd.DataFrame(ddos['ID'].unique())
    port_id = pd.DataFrame(port['ID'].unique())
    bot_id = pd.DataFrame(bot['ID'].unique())
    nor_id.to_csv('/home/usami/TBN64/dataset_pre/CICDDoS2017/TrafficLabelling/Friday-WorkingHours-nor.csv', index=False)
    ddos_id.to_csv('/home/usami/TBN64/dataset_pre/CICDDoS2017/TrafficLabelling/Friday-WorkingHours-ddos.csv', index=False)
    port_id.to_csv('/home/usami/TBN64/dataset_pre/CICDDoS2017/TrafficLabelling/Friday-WorkingHours-port.csv', index=False)
    bot_id.to_csv('/home/usami/TBN64/dataset_pre/CICDDoS2017/TrafficLabelling/Friday-WorkingHours-bot.csv', index=False)


/home/usami/miniconda3/envs/human_action/lib/python3.8/site-packages/scapy/layers/ipsec.py:469: CryptographyDeprecationWarning: Blowfish has been deprecated and will be removed in a future release
  cipher=algorithms.Blowfish,
/home/usami/miniconda3/envs/human_action/lib/python3.8/site-packages/scapy/layers/ipsec.py:483: CryptographyDeprecationWarning: CAST5 has been deprecated and will be removed in a future release
  cipher=algorithms.CAST5,


In [1]:
import subprocess
import os
import hashlib
import numpy as np
from tqdm import tqdm
import pandas as pd

folder_path = '/home/usami/TBN64/dataset_pre/CICDDoS2017/Friday-WorkingHours'
TIMEOUT = 30
cnt_normal = 0

def read_pcap_with_tshark(file_path):
    command = [
        'tshark', '-r', file_path, '-T', 'fields',
        '-e', 'frame.time_epoch',
        '-e', 'ip.src', '-e', 'ip.dst',
        '-e', 'ip.proto',
        '-e', 'tcp.srcport', '-e', 'tcp.dstport', '-e', 'tcp.flags',
        '-e', 'udp.srcport', '-e', 'udp.dstport',
        '-e', 'icmp.type',
        '-e', 'frame.len', '-e', 'ip.flags', '-e', 'ip.ttl'
    ]
    process = subprocess.Popen(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in process.stdout:
        yield line.decode().strip().split('\t')

for filename in range(0, 50):
    file_path = folder_path + '/Friday__000' + str(filename).zfill(2) + '.pcap'
    cnt_normal = 0
    if os.path.isfile(file_path):
        print(file_path)
        flows = {}
        flows_cnt = {}

        for packet in tqdm(read_pcap_with_tshark(file_path)):
            if len(packet) < 12:
                continue
            
            timestamp, src_ip, dst_ip, protocol_numbers, tcp_sport, tcp_dport, tcp_flags, udp_sport, udp_dport, icmp_type, pkt_len, ip_flags, ttl = packet
            proto_layer = None
            flow_id_str = None
            
            for protocol_number in protocol_numbers.split(','):
                protocol_number = int(protocol_number)
                
                if tcp_sport and tcp_dport:
                    proto_layer = 'TCP'
                    flow_id_str = f"{src_ip}:{tcp_sport}->{dst_ip}:{tcp_dport}:{protocol_number}"
                elif udp_sport and udp_dport:
                    proto_layer = 'UDP'
                    flow_id_str = f"{src_ip}:{udp_sport}->{dst_ip}:{udp_dport}:{protocol_number}"
                elif icmp_type:
                    proto_layer = 'ICMP'
                    flow_id_str = f"{src_ip}->{dst_ip}:{protocol_number}"
                
                if flow_id_str:
                    if nor['ID'].isin([flow_id_str]).sum().values or ddos['ID'].isin([flow_id_str]).sum().values:
                        flow_id_hash = int(hashlib.md5(flow_id_str.encode()).hexdigest(), 32)
                        if flow_id_hash not in flows_cnt:
                            flows_cnt[flow_id_hash] = 0
                        if proto_layer == 'TCP':
                            if int(tcp_flags, 16) & 0x02 and not int(tcp_flags, 16) & 0x10:  # SYN flag set and ACK flag not set
                                flows_cnt[flow_id_hash] += 1
                        flow_id = int(hashlib.md5((flow_id_str + ':' + str(flows_cnt[flow_id_hash])).encode()).hexdigest(), 32)
                        if flow_id in flows:
                            last_packet_time = flows[flow_id][-1]['timestamp']
                            if (float(timestamp) - last_packet_time) > TIMEOUT:
                                flows_cnt[flow_id_hash] += 1
                                flow_id = int(hashlib.md5((flow_id_str + ':' + str(flows_cnt[flow_id_hash])).encode()).hexdigest(), 32)

                        if flow_id:
                            sorted_flow_id = flow_id
                            if sorted_flow_id not in flows:
                                flows[sorted_flow_id] = []
                            packet_info = {
                                'id': flow_id_str,
                                'timestamp': float(timestamp),
                                'time': 0,
                                'proto': protocol_number,
                                'pkt_len': int(pkt_len),
                                'ip_flag': int(ip_flags, 16) if ip_flags else 0,
                                'ttl': int(ttl, 16) if ttl else 0,
                                'tcp_len': None,
                                'tcp_ack': None,
                                'tcp_flag': int(tcp_flags, 16) if tcp_flags else 0,
                                'udp_len': None,
                                'icmp_type': int(icmp_type, 16) if icmp_type else 0,
                                'label': 2  # 0: Normal, 1: DoS, 2: DDoS
                            }

                            if proto_layer == 'TCP':
                                packet_info['tcp_len'] = int(tcp_sport, 16)
                            elif proto_layer == 'UDP':
                                packet_info['udp_len'] = int(udp_sport, 16)
                            
                            if flows[sorted_flow_id]:
                                first_packet_time = flows[sorted_flow_id][0]['timestamp']
                                packet_info['time'] = float(timestamp) - first_packet_time
                            if packet_info['time'] == 0:
                                if nor['ID'].isin([flow_id_str]).sum().values:
                                    packet_info['label'] = 0
                                    cnt_normal += 1
                            else:
                                packet_info['label'] = flows[sorted_flow_id][0]['label']
                            flows[sorted_flow_id].append(packet_info)

        flow_arrays = list(flows.values())
        for flow_id in flows:
            for packet in flows[flow_id]:
                del packet['timestamp']
        print(f"Số lượng các luồng Normal: {cnt_normal}")
        print(f"Số lượng các luồng: {len(flow_arrays)}")
        folroot = os.path.dirname(folder_path)
        for folname in [folroot+'/data',
                        folroot+'/label']:
            if not os.path.exists(folname):
                    os.makedirs(folname)
        output_file_path = os.path.dirname(os.path.dirname(file_path))+'/data/'+file_path.split('/')[-1].replace('.pcap','.json')
        data = []
        lb = []
        time_window = 60
        h_window = 32
        e = time_window / h_window
        
        for flow_id in flows:
            data_array = []
            flow = flows[flow_id]
            for h in range(h_window):
                data_array_t = []
                for record in flow:
                    if (record['time'] < (h + 1) * e) and (record['time'] >= h * e):
                        record_list = [
                            record['proto'],
                            record['pkt_len'],
                            record['ip_flag'],
                            record['ttl'],
                            record['tcp_len'] if record['tcp_len'] is not None else 0,
                            record['tcp_ack'] if record['tcp_ack'] is not None else 0,
                            record['tcp_flag'] if record['tcp_flag'] is not None else 0,
                            record['udp_len'] if record['udp_len'] is not None else 0,
                            record['icmp_type'] if record['icmp_type'] is not None else 0,
                        ]
                        data_array_t.append(record_list)
                if len(data_array_t) == 0 or np.isnan(np.mean(data_array_t, axis=0)).any():
                    data_array_t_mean = np.zeros((1, 9))
                else:
                    data_array_t_mean = np.mean(data_array_t, axis=0)
                data_array.append(data_array_t_mean)
            data.append(np.vstack(data_array))
            lb.append(record['label'])
        
        np.save(output_file_path.replace('.json', '.npy'), data, allow_pickle=True)
        np.save(output_file_path.replace('.json', 'labels.npy'), lb, allow_pickle=True)
        break


In [ ]:
import os
import numpy as np
import gc
import h5py


def unique_rows(matrix):
    seen = set()
    unique_indices = []
    
    for i, row in enumerate(matrix):
        row_hash = hash(row.tobytes())
        if row_hash not in seen:
            seen.add(row_hash)
            unique_indices.append(i)
    unique_matrix = matrix[unique_indices]
    return unique_matrix
def round_to_n_significant_figures(tensor, n):
    """
    Efficiently round each element in a NumPy array to n significant figures.
    Handles edge cases to avoid NaN values.
    """
    tensor = np.asarray(tensor)
    # Handle zero separately to avoid division by zero
    non_zero_mask = tensor != 0
    abs_tensor = np.abs(tensor[non_zero_mask])
    order_of_magnitude = np.floor(np.log10(abs_tensor))
    scale = np.power(10, n - 1 - order_of_magnitude)
    tensor[non_zero_mask] = np.round(tensor[non_zero_mask] * scale) / scale
    return tensor
for h_window in [64,32,16,8]:
    datala_list = []
    time_window = 1
    folder_root = '/mnt/d/dataset_pre/CICDDoS2017'
    folder_path = folder_root+f'/h{str(h_window)}b/time_window_{str(time_window)}s/data'
    cnt = 0
    cnt_file = 0
    for i in range(1000):
        # if i in range(0,50) or i in range(214,283):
        if 1:
            filename = str(i) + '.npy'
            if os.path.isfile(os.path.join(folder_path, filename)):
                if(cnt%10==0 and cnt !=0):
                    print(cnt)
                if(cnt%100==0 and cnt !=0):
                    datalas = np.vstack(datala_list)
                    datalas = round_to_n_significant_figures(datalas,3) 
                    datala_t = unique_rows(datalas)
                    data = datala_t[:, :-1].reshape(-1, h_window, 9).astype(np.float32)
                    labels = datala_t[:, -1].astype(int)

                    np.save(folder_root+f'/{str(time_window)}{str(h_window)}_{str(cnt_file)}_data.npy', data)
                    np.save(folder_root+f'/{str(time_window)}{str(h_window)}_{str(cnt_file)}_labels.npy', labels)
                    del datalas
                    del datala_t
                    del data
                    del labels
                    gc.collect()
                    datala_list = []
                    cnt_file+=1
                file_path = os.path.join(folder_path, filename)
                data = np.load(file_path)
                labels = np.load(file_path.replace('/data/', '/labels/'))
                data = data.reshape(data.shape[0], -1).reshape(data.shape[0], -1)
                labels = labels.reshape(-1, 1)

                datala_e = np.hstack((data, labels))
                datala_list.append(datala_e)
                cnt+=1
                del data
                del labels
                del datala_e
                gc.collect()

    datalas = np.vstack(datala_list)
    datala_t = unique_rows(datalas)
    data = datala_t[:, :-1].reshape(-1, h_window, 9).astype(np.float32)
    labels = datala_t[:, -1].astype(int)

    np.save(folder_root+f'/{str(time_window)}{str(h_window)}_{str(cnt_file)}_data.npy', data)
    np.save(folder_root+f'/{str(time_window)}{str(h_window)}_{str(cnt_file)}_labels.npy', labels)
    del data
    del labels
    del datala_t
    del datalas
    gc.collect()
    datala_list = []
    datala_list_end = []
    tt = cnt_file+1
    cnt_file = 0
    for cnt_file in range(tt):
        print(cnt_file)
        if os.path.isfile(folder_root+f'/{str(time_window)}{str(h_window)}_{str(cnt_file)}_data.npy'):
            data = np.load(folder_root+f'/{str(time_window)}{str(h_window)}_{str(cnt_file)}_data.npy')
            labels = np.load(folder_root+f'/{str(time_window)}{str(h_window)}_{str(cnt_file)}_labels.npy')
            os.remove(folder_root+f'/{str(time_window)}{str(h_window)}_{str(cnt_file)}_data.npy')
            os.remove(folder_root+f'/{str(time_window)}{str(h_window)}_{str(cnt_file)}_labels.npy')

            data = data.reshape(data.shape[0], -1).reshape(data.shape[0], -1)
            labels = labels.reshape(-1, 1)
            datala_e = np.hstack((data, labels))
            datala_list.append(datala_e)
            del data
            del labels
            del datala_e
            gc.collect()

    datalas = np.vstack(datala_list)
    data = datalas[:, :-1].reshape(-1, h_window, 9).astype(np.float32)
    labels = datalas[:, -1].astype(int)
    del datala_list_end
    gc.collect()
    with h5py.File(folder_root + f'/{str(time_window)}{str(h_window)}_data.h5', 'w') as hf:
        hf.create_dataset('data', data=data)

    with h5py.File(folder_root + f'/{str(time_window)}{str(h_window)}_labels.h5', 'w') as hf:
        hf.create_dataset('labels', data=labels)
